In [39]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim.sgd import SGD
from torch.nn.parameter import Parameter
from IPython.display import display
import matplotlib.pyplot as plt
import tqdm

In [8]:
# 定义数据集
class my_dataset(Dataset):
    def __init__(self):
        super(my_dataset,self).__init__()
        self.path = '../../data/z=wxwy.npy'
        self.data = np.load(self.path)
    def __getitem__(self, index):
        return np.array(self.data[index],dtype=np.float32)
    
    def __len__(self):
        return self.data.shape[0]

In [71]:
# 定义模型
class my_model(nn.Module):
    def __init__(self,dim_hidden=1):
        super(my_model,self).__init__()
        self.linear1 = nn.Linear(2,dim_hidden,bias=False)
        self.linear2 = nn.Linear(dim_hidden,dim_hidden,bias=False)
        self.linear3 = nn.Linear(dim_hidden,1,bias=False)
        self.relu = nn.ReLU(inplace=True)
        self.gelu = nn.GELU()
        self.sigmoid = nn.Sigmoid()
        # self.bn0 = nn.BatchNorm1d(2)
        self.bn1 = nn.BatchNorm1d(dim_hidden)
        self.bn2 = nn.BatchNorm1d(dim_hidden)
    def forward(self, x):
        # x = self.bn0(x)
        x = self.linear1(x)
        x = self.bn1(x)
        self.relu(x)
        # x = self.gelu(x)
        # x = self.sigmoid(x)
        # x = self.linear2(x)
        # x = self.bn2(x)
        # self.relu(x)
        # x = self.gelu(x)
        # x = self.sigmoid(x)
        x = self.linear3(x)
        return x

In [65]:
class Loss(nn.Module):
    def __init__(self):
        super(Loss, self).__init__()
    
    def forward(self, x, y):
        distance = 1/2 * (y - x) ** 2
        loss = torch.sum(distance)
        loss_mean = loss / x.shape[0]
        return loss_mean

In [110]:
batch_size = 10000
num_workers = 0
lr = 0.005
epochs = 10000

In [119]:
my_data = my_dataset()
my_dataloader = DataLoader(my_data,batch_size,num_workers=num_workers,drop_last=True,shuffle=True)
model = my_model(50)
# init_weight = torch.tensor([[0.5455, 0.0551]])
# init_weight = torch.tensor([[0, -9.]])
# model.linear.weight = Parameter(init_weight)
loss_f = Loss()
# 定义优化器
optimizer = SGD(model.parameters(),lr=lr)

In [120]:

for data in my_dataloader :
    data = data
    pass
pbar = tqdm.tqdm(range(epochs))
for epoch in pbar:
    output = model(data[:,:-1])
    loss = loss_f(output, data[:,-1:])
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    pbar.set_description("Processing %.4f" % loss.item())
    
        # tqdm.tqdm.set_postfix(loss='{:.6f}'.format(loss.item()))#后缀设置一些想要的监控信息


Processing 0.0894: 100%|██████████| 10000/10000 [01:17<00:00, 128.87it/s]
